<a href="https://colab.research.google.com/github/Juhwan01/DeepDive/blob/main/transformer%EC%82%AC%EC%9A%A9_%EB%84%A4%EC%9D%B4%EB%B2%84%EB%A6%AC%EB%B7%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 새 셀 첫 번째 줄
# TF_USE_LEGACY_KERAS='1' 설정하면 옛날 방식으로 돌아가서 모든 게 정상 작동
import os
os.environ['TF_USE_LEGACY_KERAS'] = '1'

In [ ]:
import pandas as pd
import numpy as np
import urllib.request
import os
from tqdm import tqdm
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel

In [ ]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt",
                          filename="ratings_train.txt")

urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt",
                          filename="ratings_test.txt")

In [ ]:
test_data=pd.read_table('ratings_test.txt')
train_data=pd.read_table('ratings_train.txt')

In [ ]:
print('훈련용 리뷰 개수 :', len(train_data))
print('테스트용 리뷰 개수 :', len(test_data))

In [ ]:
train_data.drop_duplicates(subset=['document'],inplace=True)
train_data=train_data.dropna(how='any')
print('훈련 데이터의 리뷰 수 :',len(train_data))

In [ ]:
test_data=test_data.dropna(how='any')
print('테스트 데이터의 리뷰 수 :',len(test_data))

In [ ]:
# 사전학습된 토크나이저 불러오기
# Hugging Face transformers 라이브러리에서 제공하는,Rust 언어로 구현된 초고속 토크나이저
from transformers import BertTokenizerFast
tokenizer=BertTokenizerFast.from_pretrained('klue/bert-base')

In [ ]:
#.values는 DataFrame에서 열을 numpy 배열로 가져오고,
#.tolist()는 numpy 배열을 파이썬 리스트로 변환합니다.
# 하지만 최신 파이썬에서는 tolist만써도 충분함
# 머신러닝·딥러닝 모델, 전처리 도구, 토크나이저 등은 대부분 리스트 또는 배열 형태의 데이터를 입력으로 요구합니다
X_train_list = train_data['document'].tolist()
X_test_list = test_data['document'].tolist()
y_train = train_data['label'].tolist()
y_test = test_data['label'].tolist()

In [ ]:
# truncation = True -> Bert의 기본 max_length=512를 초과하는 경우에 뒤를 자른다
# padding = True -> 짧은 문장 0으로 패딩
X_train = tokenizer(X_train_list, truncation=True, padding=True)
X_test = tokenizer(X_test_list, truncation=True, padding=True)

In [ ]:
# 샘플에 .tokens를 사용하여 토큰화 결과 확인
print(X_train[0].tokens)

In [ ]:
# 샘플에 .ids를 하면 정수 인코딩 결과 확인
print(X_train[0].ids)

In [ ]:
# 세그먼트 인코딩 값 확인 모든 위치 값이 0 -> 인식할 문장 종류 1개
print(X_train[0].type_ids)

In [ ]:
# 어텐션 마스크를 통해 실제 단어가 있는 위치와 패딩의 위치를 확인
print(X_train[0].attention_mask)


# 📘 `tf.data.Dataset.from_tensor_slices()` 총정리

## ✅ 개념 요약

### 📌 한 줄 설명
> 샘플 하나하나로 나눠서 `tf.data.Dataset` 객체를 만들어주는 함수

### 🔧 언제 써?
TensorFlow 모델(`model.fit()`)에 학습용 데이터를 넣기 위한 Dataset 객체 생성할 때 사용

## ✅ 기본 사용법

```python
dataset = tf.data.Dataset.from_tensor_slices((X, y))
````

* `X`: 입력 데이터 (넘파이 배열, 텐서, 딕셔너리 등 가능)
* `y`: 정답(레이블)

## ✅ 작동 방식 (샘플 단위로 쪼개짐)

```python
X = [[1, 2], [3, 4], [5, 6]]
y = [0, 1, 0]
dataset = tf.data.Dataset.from_tensor_slices((X, y))
```

➡️ 위 코드는 아래처럼 분할됨:

* 샘플 1: `([1, 2], 0)`
* 샘플 2: `([3, 4], 1)`
* 샘플 3: `([5, 6], 0)`

## ✅ `dict(X_train)`이 필요한 경우

| `X_train` 자료형      | dict로 바꿔야 하나? | 이유                 |
| ------------------ | ------------- | ------------------ |
| `pandas.DataFrame` | ✅ **필요함**     | 안 바꾸면 열 이름 정보가 사라짐 |
| `dict`             | ❌ 필요 없음       | 이미 딕셔너리니까 그대로 사용   |
| `numpy.ndarray`    | ❌ 필요 없음       | 그대로 사용 가능          |
| `Tensor`           | ❌ 필요 없음       | 그대로 사용 가능          |

## ✅ 예제 모음

### 1. `DataFrame`일 때 → dict 필요함

```python
import pandas as pd
X_train = pd.DataFrame({'a': [1, 2], 'b': [3, 4]})
y_train = [0, 1]

dataset = tf.data.Dataset.from_tensor_slices((dict(X_train), y_train))
```

### 2. `numpy.ndarray`일 때

```python
import numpy as np
X_train = np.array([[1, 3], [2, 4]])
y_train = np.array([0, 1])

dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
```

### 3. `dict`일 때

```python
X_train = {'a': [1, 2], 'b': [3, 4]}
y_train = [0, 1]

dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
```

## ✅ 그 다음은 보통 이렇게 사용함

```python
dataset = dataset.shuffle(buffer_size=100).batch(32)
model.fit(dataset, epochs=5)
```

* `.shuffle()`: 데이터 섞기
* `.batch()`: 배치로 묶기
* `model.fit()`: 학습 시작

## 📌 핵심 요약

| 질문                        | 답변                            |
| ------------------------- | ----------------------------- |
| `from_tensor_slices`는 뭐야? | 샘플 단위로 잘라서 Dataset 만드는 함수     |
| 왜 써?                      | TensorFlow 모델 학습용 데이터셋 만들기    |
| dict(X\_train)은 언제 써?     | `X_train`이 DataFrame이면 꼭 써야 함 |
| dict 말고도 입력 가능한 형태는?      | dict, ndarray, tensor 모두 가능   |




In [ ]:
import tensorflow as tf
from transformers import TFBertForSequenceClassification
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
# tf.data.Dataset.from_tensor_slices() -> 샘플 하나하나로 나눠서 tf.data.Dataset 객체를 만들어주는 함수.
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(X_train),
    y_train
))

In [ ]:
val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(X_test),
    y_test
))

In [ ]:
# optimizer -> Adam 선택
# optimizer를 선택하지 않을 경우 BERT 출력만 나오기 때문에 model(input_ids)로 예측 시 단순 벡터만 출력되고,
# Dense(num_labels) 같은 분류용 레이어를 별도로 연결해줘야 합니다.
# 버전을 보니 TensorFlow 2.18.0 + Keras 3.8.0 조합에서 발생하는 문제가 있음. Keras 3.x는 TensorFlow와 독립적으로 동작하면서 호환성 이슈가 발생
# tf.optimizers 사용 (tf.keras.optimizers 아님)
optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=5e-5)

In [ ]:
#  PyTorch용 모델 가중치만 존재하는 경우 또는 PyTorch로 저장된 체크포인트를 TensorFlow 모델에 불러오고 싶다면, from_pt=True를 명시해야함
model = TFBertForSequenceClassification.from_pretrained("klue/bert-base", num_labels=2, from_pt = True)
# hf_compute_loss -> 이 메서드는 모델에 맞는 손실 함수를 자동으로 계산
# hf_compute_loss = 모델 내장 손실 함수 사용
model.compile(optimizer=optimizer, loss=model.hf_compute_loss,metrics=['accuracy'])

In [ ]:
# 조기 종료 설정 -> 여기서도 위에 optimizer객체 사용할때도 버전문제 발생으로 맨 윗줄에
# 레거시 모드 사용으로 변경하는 시점
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)
model.fit(
    train_dataset.shuffle(10000).batch(32),  # 여기서 배치 처리!
    epochs=2,
    # batch_size=32,  # 이 줄은 제거 (Dataset에서는 무시됨)
    validation_data=val_dataset.shuffle(10000).batch(32),  # 검증 데이터도 배치 처리
    callbacks=[early_stopping]
)

In [ ]:
# 테스트 데이터를 바탕으로 평가
model.evaluate(val_dataset.batch(1024))

In [ ]:
# 학습된 BERT 모델을 'nsmc_model/bert-base' 경로에 저장합니다.
# save_pretrained()는 모델의 가중치와 설정 파일을 함께 저장합니다.
model.save_pretrained('nsmc_model/bert-base')

# 토크나이저도 같은 경로에 저장합니다.
# 모델과 함께 반드시 저장해야 나중에 동일한 토크나이저로 재사용할 수 있습니다. -> 혹시나 버전 차이로 인하여 오류 일으킬 수도 있음 내가 아까 만난 오류 또 만날 가능성
tokenizer.save_pretrained('nsmc_model/bert-base')

In [ ]:
from transformers import TextClassificationPipeline

In [ ]:
# 사전에 학습해둔 토크나이저랑 모델 불러오기
loaded_tokenizer = BertTokenizerFast.from_pretrained('nsmc_model/bert-base')
loaded_model = TFBertForSequenceClassification.from_pretrained('nsmc_model/bert-base')

In [ ]:
# TextClassificationPipeline 생성
# 이 파이프라인은 텍스트 분류 과정을 간단하게 처리해주는 도구입니다.
# 내부적으로 다음 작업을 자동으로 수행합니다:
# ① 토큰화 → ② 모델 추론 → ③ 결과 점수 반환
text_classifier = TextClassificationPipeline(
    tokenizer = loaded_tokenizer,   # 입력 텍스트를 토큰 ID로 변환할 토크나이저
    model = loaded_model,           # 사전 학습된 텍스트 분류 모델
    framework = 'tf',               # TensorFlow 기반 모델임을 명시 (PyTorch일 경우 'pt')
    return_all_scores = True        # 모든 클래스(label)에 대한 점수를 반환 (False일 경우 최고 점수 label만 반환)
)
# 예를 들어 이진 분류일 경우 LABEL_0, LABEL_1 모두에 대한 score를 반환합니다.

In [ ]:
# 한 문장을 텍스트 분류 파이프라인에 넣습니다.
# return_all_scores=True 이므로 결과는 다음과 같은 2중 리스트 구조입니다:
# [[{'label': 'LABEL_0', 'score': 0.85}, {'label': 'LABEL_1', 'score': 0.15}]]
# → 바깥 리스트는 문장 수, 안쪽 리스트는 각 레이블의 점수
result = text_classifier("뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아")

# 첫 번째 문장에 대한 결과만 가져옵니다. (문장 1개만 넣었으므로 [0]으로 추출)
first_result = result[0]

print(first_result)
# 출력: [{'label': 'LABEL_0', 'score': ...}, {'label': 'LABEL_1', 'score': ...}]